# Make all in one package

milsymbol.js: <https://github.com/spatialillusions/milsymbol/releases/tag/v2.2.0>

In [1]:
import json
import pathlib
import rapidfuzz
from py_mini_racer import MiniRacer


class SIDCFuzzySearcher:
    def __init__(self, path_to_set_a: pathlib.Path, path_to_set_b: pathlib.Path, path_to_milsymbolsjs: pathlib.Path):
        self.score_cutoff = 70  # threshold for rapidfuzz
        # you may change it for your needs
        #    version 1.0, reality, unknown, unknown, present, unknown, unknown
        self.defaults_set_a = {'1': '1', '2': '0', '3': '0', '4': '1', '56': '00', '7': '0', '8': '0', '910': '00'}
        self._data_a = self._load_json(path_to_set_a)
        self._data_b = self._load_json(path_to_set_b)
        self._ctx = self._load_js(path_to_milsymbolsjs)

    def _load_json(self, path: pathlib.Path) -> dict:
        with open(path, 'r') as fp:
            return json.load(fp)

    def _load_js(self, path: pathlib.Path) -> MiniRacer:
        # get script from <https://github.com/spatialillusions/milsymbol/releases/tag/v2.2.0>
        with open(path, 'r') as fp:
            txt = fp.read()
        ctx = MiniRacer()
        ctx.eval(txt)
        return ctx
        
    def _search_a(self, query: str, n=1, show_results=False) -> str:
        """ query in set A treated like few separate words
        """
        choices_a = self._data_a.keys()
        # try to find each word separately, get uniq results
        findings = set([self._fuzzy_search(q, choices_a, n, show_results) for q in query.split()])
        # update default values
        answer_a = self.defaults_set_a.copy()
        # '3.Reality' is a key to self._data_a where '3' is a idx of set_a
        #   self._data_a['3.Reality'] -> '0' 
        answer_a.update({f.split('.')[0]: self._data_a[f] for f in findings if f})
        # format an answer string
        a = f"{answer_a['1']}{answer_a['2']}{answer_a['3']}{answer_a['4']}{answer_a['56']}{answer_a['7']}{answer_a['8']}{answer_a['910']}"
        return a
    
    def _search_b(self, query: str, mod1: str = '', mod2: str = '', n=1, show_results=False) -> str:
        """ query in set B treated like a single sentence
            each modifier is a separate single word/sentence
        """
        choices = self._data_b.keys()
        # try to find entity
        #   'Land unit.Fires.Mortar.Armored/Mechanized/Tracked'
        selected_key = self._fuzzy_search(query, choices, n, show_results)
        #   '130801'
        answer_b = self._data_b[selected_key] if selected_key else '000000'
        #   'Land unit' -- prefix of selected_key
        selected_b = selected_key.split('.')[0] if selected_key else selected_key
        # try to find modifiers
        answer_mod1 = self._search_b_mode(mod1, selected_b, suffix='.modifier_1', show_results=show_results)
        answer_mod2 = self._search_b_mode(mod2, selected_b, suffix='.modifier_2', show_results=show_results)
        # format an answer string
        b = answer_b + answer_mod1 + answer_mod2
        return b

    def _search_b_mode(self, query: str | None, selected_b: str | None, suffix='.modifier_1', n=1, show_results=False) -> str:
        if query and selected_b:
            # ['Land unit.modifier_1.Attack', ..]
            choices = [k for k in self._data_b.keys() if selected_b + suffix in k]
            # 'Land unit.modifier_1.Attack'
            x = self._fuzzy_search(query, choices, n, show_results)
            # '03'
            answer_mod = self._data_b[x] if x else '00'
        else:
            answer_mod = '00'
        return answer_mod

    def _fuzzy_search(self, query: str, choices: list[str], n: int, show_results: bool) -> str:
        # TODO score_cutoff as a parameter
        x = rapidfuzz.process.extract(query, choices, limit=n, score_cutoff=self.score_cutoff, scorer=rapidfuzz.fuzz.partial_ratio)
        if show_results:
            print(f"{query}:")
            if x:
                for i in x:
                    print(f'\t{i}')
            else:
                print('\tNOTHING')
        x = x[0][0] if x else ''
        return x
   
    def get_sidc(self, query_a='', query_b='', mod1='', mod2='', show_results=False) -> str:
        """ query in set A treated like few separate words
            query in set B treated like a single sentence
            each modifier is a separate single word/sentence
        """
        a = self._search_a(query_a, show_results=show_results)
        b = self._search_b(query_b, mod1, mod2, show_results=show_results)
        return a + b

    def get_svg(self, sidc: str, size=35) -> str:
        svg_text = self._ctx.eval(f'new ms.Symbol({sidc}, {{"size": {size}}}).asSVG()')
        return svg_text

    def show_top_n(self, query, n=10) -> None:
        print('IN SET A -- ', end='')
        choices_a = self._data_a.keys()
        _ = self._fuzzy_search(query, choices_a, n, True)
        print('\n', end='')
        print('IN SET B -- ', end='')
        choices_b = self._data_b.keys()
        _ = self._fuzzy_search(query, choices_b, n, True)

## Test 2525D

In [2]:
# you need to download milsynbol.js
x = SIDCFuzzySearcher('../fuzzy_sidc/set_a.json', '../fuzzy_sidc/set_b_2525d.json', '../fuzzy_sidc/milsymbol.js')

query_a = "Hostile Realty Land Present Platoon TaskForce"
query_b = "mortar armore"
x.get_sidc(query_a=query_a, query_b=query_b, mod1='sniper', mod2='airborn', show_results=True)

Hostile:
	('4.Hostile/Faker', 100.0, 11)
Realty:
	('3.Reality', 83.33333333333334, 2)
Land:
	('56.Land Unit', 100.0, 17)
Present:
	('7.Present', 100.0, 35)
Platoon:
	('910.Platoon/detachment', 100.0, 53)
TaskForce:
	('8.Task Force', 88.88888888888889, 45)
mortar armore:
	('Land unit.Fires.Mortar.Armored/Mechanized/Tracked', 76.92307692307692, 1566)
sniper:
	('Land unit.modifier_1.Sniper', 90.9090909090909, 59)
airborn:
	('Land unit.modifier_2.Airborne', 85.71428571428572, 1)


'10061004141308016101'

In [3]:
x.get_svg('10061004141308016101')

'<svg xmlns="http://www.w3.org/2000/svg" version="1.2" baseProfile="tiny" width="53.2" height="67.2" viewBox="24 -16 152 192"><path d="M 100,28 L172,100 100,172 28,100 100,28 Z" stroke-width="4" stroke="black" fill="rgb(255,128,128)" fill-opacity="1" ></path><circle cx="100" cy="115" r="5" stroke-width="4" stroke="black" fill="none" ></circle><path d="M100,111 l0,-30 M90,90 l10,-10 10,10" stroke-width="4" stroke="black" fill="none" ></path><path d="M 70,120 l 60,0 c10,0 10,10 0,10 l -60,0 c-10,0 -10,-10 0,-10" stroke-width="4" stroke="black" fill="none" ></path><path d="m 120,65 -11,0 m 11,10 -14,0 m 4,-14 -30,0 0,18 25,0 z m 10,2 0,14" stroke-width="4" stroke="black" fill="none" ></path><path d="M55,28 L55,-12 145,-12 145,28" stroke-width="4" stroke="black" fill="none" ></path><g transform="translate(0,0)" stroke-width="4" stroke="black" fill="none" ><circle cx="100" cy="8" r="7.5" fill="black" ></circle><circle cx="70" cy="8" r="7.5" fill="black" ></circle><circle cx="130" cy="8" r="

In [4]:
x.score_cutoff = 77
x.show_top_n('airborne')

IN SET A -- airborne:
	NOTHING

IN SET B -- airborne:
	('Land unit.modifier_2.Airborne', 93.33333333333333, 1791)
	('Control Measures.Maneuver Areas.Axis of Advance.Friendly Airborne/Aviation', 87.5, 79)
	('Control Measures.Airspace Control Points.Airborne Early Warning (AEW) Station', 87.5, 130)
	('Signals intelligence.modifier_1.Airborne Search and Bombing', 87.5, 1363)
	('Signals intelligence.modifier_1.Airborne Intercept', 87.5, 1364)
	('Signals intelligence.modifier_1.Airborne Reconnaissance and Mapping', 87.5, 1366)
	('Air.Military.Fixed Wing.Airborne Command Post (ACP)', 87.5, 2168)
	('Air.Military.Fixed Wing.Airborne Early Warning (AEW)', 87.5, 2169)
	('Air.modifier_1.Airborne Command Post (ACP)', 87.5, 2215)
	('Air.modifier_1.Airborne Early Warning (AEW)', 87.5, 2216)


## Test APP6D

In [5]:
# you need to download milsynbol.js
x = SIDCFuzzySearcher('../fuzzy_sidc/set_a.json', '../fuzzy_sidc/set_b_app6d.json', '../fuzzy_sidc/milsymbol.js')

query_a = "Hostile Realty Land Present Platoon TaskForce"
query_b = "mortar armore"
x.get_sidc(query_a=query_a, query_b=query_b, mod1='sniper', mod2='airborn', show_results=True)

Hostile:
	('4.Hostile/Faker', 100.0, 11)
Realty:
	('3.Reality', 83.33333333333334, 2)
Land:
	('56.Land Unit', 100.0, 17)
Present:
	('7.Present', 100.0, 35)
Platoon:
	('910.Platoon/detachment', 100.0, 53)
TaskForce:
	('8.Task Force', 88.88888888888889, 45)
mortar armore:
	('Land unit.Fires.Mortar.Armoured/Mechanized/ Tracked', 76.92307692307692, 1352)
sniper:
	('Land unit.modifier_1.Sniper', 90.9090909090909, 61)
airborn:
	('Land unit.modifier_2.Airborne', 85.71428571428572, 1)


'10061004141308016101'

In [6]:
x.score_cutoff = 50
x.show_top_n('airborne')

IN SET A -- airborne:
	('3.Simulation', 61.53846153846154, 4)
	('56.Land Civilian Unit/Organization', 61.53846153846154, 18)
	('56.Land Installation', 61.53846153846154, 20)
	('910.Battalion/squadron', 61.53846153846154, 55)
	('910.Wheeled and tracked combination', 61.53846153846154, 67)
	('56.Mine Warfare', 57.14285714285714, 24)
	('910.Section', 54.54545454545454, 52)
	('910.Division', 54.54545454545454, 58)
	('7.Present/Fully capable', 50.0, 37)
	('8.Task Force', 50.0, 45)

IN SET B -- airborne:
	('Land unit.modifier_2.Airborne', 93.33333333333333, 1565)
	('Dismounted individual.modifier_2.Airborne', 93.33333333333333, 2006)
	('Control Measures.Manoeuvre Areas.Axis of Advance.Friendly Airborne/Aviation', 87.5, 95)
	('Control Measures.Airspace Control Points.Airborne Early Warning (AEW) Station', 87.5, 153)
	('Air.Military.Fixed Wing.Airborne Command Post (ACP)', 87.5, 2063)
	('Air.Military.Fixed Wing.Airborne Early Warning (AEW)', 87.5, 2064)
	('Air.modifier_1.Airborne Command Post 